# Assignment 1 - Data Science in Finance and Insurance
Submission by: Dennis Goldenberg


In [3]:
#import statements
import scipy as sc;
import numpy as np;
import matplotlib as plt;


## Generating Sample from $\chi^2(11)$ random variable

In [5]:
#Draw from chi-squared dist
samples = sc.stats.chi2.rvs(df = 11, size = 2000)

array([ 4.45040833,  9.25093969, 14.88020065, ...,  8.69853293,
        8.56054811, 14.12088397])